## MINIMUM EDIT DISTANCE
example from 'Speech and Language Processing An Introduction to Natural Language Processing, Computational Linguistics, and Speech Recognition' Jurafsky and Martin 3rd edition

Same letters have **cost 0** so, if and only if source[i] != target [j]:

$
\qquad\displaystyle d_{ij} = 
  \min \begin{cases} 
         d_{i-1, j}  + c_\mathrm{del}(source_{i}) \\ 
         d_{i,j-1}   + c_\mathrm{ins}(target_{j}) \\ 
         d_{i-1,j-1} + [a_j \neq b_i] \cdot c_\mathrm{sub}(target_{j}, source_{i}) \end{cases}
$

In [ ]:
#source = 'Stanford President Marc Tessier-Lavigne'
#target = 'Stanford University President Marc Tessier-Lavigne'
source = 'sittenn'
target = 'kittgen'

In [ ]:
import numpy

# Costs for the operations
INS_COST = 1
DEL_COST = 1
SUB_COST = 1

def minimum_edit_distance(source, target) :

    # Create a dp matrix of dimension (source + 1) x (target + 1)
    # So we have (target +1) rows and (source +1) columns
    dp = [[0] * (len(source) + 1) for i in range(len(target) + 1)]
    do = [['-'] * (len(source) + 1) for i in range(len(target) + 1)]

    # Initialize the required values of the matrix
    for i in range(1, len(target) + 1) :
        dp[i][0] = dp[i - 1][0] + INS_COST
    for i in range(1, len(source) + 1) :
        dp[0][i] = dp[0][i - 1] + DEL_COST

    # Maintain the record of opertions
    # Record is one tuple. Eg : (INSERT, 'a') or (SUBSTITUTE, 'e', 'r') or (DELETE, 'j')
    operations_performed = []

    # Build the matrix following the algorithm
    for i in range(1, len(target) + 1) :
        for j in range(1, len(source) + 1) :
            if source[j - 1] == target[i - 1] :
                dp[i][j] = dp[i - 1][j - 1]
            else :
                dp[i][j] =  min(dp[i - 1][j] + INS_COST, \
                                dp[i - 1][j - 1] + SUB_COST, \
                                dp[i][j - 1] + DEL_COST)

    # Initialization for backtracking
    i = len(target)
    j = len(source)

    # Backtrack to record the operation performed
    while (i != 0 and j != 0) :
        # If the character of the source string is equal to the character of the destination string,
        # no operation is performed
        if target[i - 1] == source[j - 1] :
            do[i][j] = 'K'
            i -= 1
            j -= 1
        else :
            # Check if the current element is derived from the upper-left diagonal element
            if dp[i][j] == dp[i - 1][j - 1] + SUB_COST :
                operations_performed.append(('S', source[j - 1], target[i - 1]))
                do[i][j] = 'S'
                i -= 1
                j -= 1
            # Check if the current element is derived from the upper element
            elif dp[i][j] == dp[i - 1][j] + INS_COST :
                operations_performed.append(('I', target[i - 1]))
                do[i][j] = 'I'
                i -= 1
            # Check if the current element is derived from the left element
            else :
                operations_performed.append(('D', source[j - 1]))
                do[i][j] = 'D'
                j -= 1
                

    # If we reach top-most row of the matrix
    while (j != 0) :
        operations_performed.append(('D', source[j - 1]))
        do[i][j] = 'D'
        j -= 1

    # If we reach left-most column of the matrix
    while (i != 0) :
        operations_performed.append(('I', target[i - 1]))
        do[i][j] = 'I'
        i -= 1

    # Reverse the list of operations performed as we have operations in reverse
    # order because of backtracking
    operations_performed.reverse()
    return dp, do, operations_performed


def print_matrix(distances, source, target):
    src = "#"+source
    tgt = "#"+target
    r0 = '    '
    ri = '- ' * (len(src)+2)
    r1 = '    '
    for i in range(len(src)):
        r1 += f'{src[i]} '
        r0 += f'{i} '
    #print(r0)
    print(ri)
    print(r1)
    print(ri)
    #src -> columns
    #tgt -> rows
    for t1 in range(len(tgt)):
        r = f'{tgt[t1]} |'
        for t2 in range(len(src)):
          r += f' {str(distances[t1][t2])}'
        print(r)
        #print(f'{token1[t1]} {int(distances[t1][t2])}', end=" ")
        print()

In [ ]:
distances, op_matrix, operations = minimum_edit_distance(source, target)

In [ ]:
print_matrix(distances, source, target)

- - - - - - - - - - 
    # s i t t e n n 
- - - - - - - - - - 
# | 0 1 2 3 4 5 6 7

k | 1 1 2 3 4 5 6 7

i | 2 2 1 2 3 4 5 6

t | 3 3 2 1 2 3 4 5

t | 4 4 3 2 1 2 3 4

g | 5 5 4 3 2 2 3 4

e | 6 6 5 4 3 2 3 4

n | 7 7 6 5 4 3 2 3



Given that:
*   'I' -> Insertion
*   'D' -> Deletion
*   'S' -> Substitution
*   'K' -> Keep previous
*   '-' -> No operations till now 


The list of operations to be performed the following:

In [ ]:
for i, op in enumerate(operations):
  print(f"({i+1}) {op}")

(1) ('S', 's', 'k')
(2) ('D', 'n')


 In the form of path on a matrix:

In [ ]:
print_matrix(op_matrix, source, target)

- - - - - - - - - - 
    # s i t t e n n 
- - - - - - - - - - 
# | - - - - - - - -

k | - S - - - - - -

i | - - K - - - - -

t | - - - K - - - -

t | - - - - K - - -

e | - - - - - K D -

n | - - - - - - - K

